# Práctica 2a 
Desarrollar una aplicación que lleve a cabo transformaciones de la imagen en
tiempo real a través de una interfaz basada en trackbars o equivalente.

- Hacer traslaciones. Es necesario indicar la magnitud de la traslación en X y en Y.
- Hacer rotaciones. Es necesario indicar el centro de giro y ángulo de giro.
- Hacer escalados uniformes y no uniformes. Es necesario indicar los factores de
escala.

In [1]:
import cv2 as cv
import numpy as np

image = cv.imread('images/cats.png')
if image is None:
    raise FileNotFoundError("No se encontró la imagen.")
h, w = image.shape[:2]

cv.namedWindow("Transformaciones")

def nothing(x):
    pass


cv.createTrackbar("Tx", "Transformaciones", w//2, w, nothing)
cv.createTrackbar("Ty", "Transformaciones", h//2, h, nothing)
cv.createTrackbar("Angulo", "Transformaciones", 0, 360, nothing)
cv.createTrackbar("Cx", "Transformaciones", w//2, w, nothing)
cv.createTrackbar("Cy", "Transformaciones", h//2, h, nothing)
cv.createTrackbar("EscalaX", "Transformaciones", 100, 200, nothing)  
cv.createTrackbar("EscalaY", "Transformaciones", 100, 200, nothing)
cv.createTrackbar("Uniforme", "Transformaciones", 0, 1, nothing)

print("Usa los sliders para transformar la imagen.")
print("Pulsa ESC para salir.")

while True:
    tx = cv.getTrackbarPos("Tx", "Transformaciones") - w//2
    ty = cv.getTrackbarPos("Ty", "Transformaciones") - h//2
    angle = cv.getTrackbarPos("Angulo", "Transformaciones")
    cx = cv.getTrackbarPos("Cx", "Transformaciones")
    cy = cv.getTrackbarPos("Cy", "Transformaciones")
    sx = cv.getTrackbarPos("EscalaX", "Transformaciones") / 100.0
    sy = cv.getTrackbarPos("EscalaY", "Transformaciones") / 100.0
    uniforme = cv.getTrackbarPos("Uniforme", "Transformaciones")

    # Traslación
    T = np.float32([[1, 0, tx],
                    [0, 1, ty]])
    img_T = cv.warpAffine(image, T, (w, h))

    # Rotación
    M_translate1 = np.float32([[1, 0, -cx],
                               [0, 1, -cy]])
    M_rotate = cv.getRotationMatrix2D((0, 0), angle, 1.0)
    M_translate2 = np.float32([[1, 0, cx],
                               [0, 1, cy]])

    M1 = np.vstack([M_translate1, [0, 0, 1]])
    M2 = np.vstack([M_rotate, [0, 0, 1]])
    M3 = np.vstack([M_translate2, [0, 0, 1]])
    M_rot = M3 @ M2 @ M1
    img_R = cv.warpAffine(img_T, M_rot[:2], (w, h))

    # Escalado
    if uniforme:
        s = sx
        S = np.float32([[s, 0, 0],
                        [0, s, 0]])
    else:
        S = np.float32([[sx, 0, 0],
                        [0, sy, 0]])

    img_final = cv.warpAffine(img_R, S, (w, h))

    img_display = img_final.copy()
    
    cv.circle(img_display, (cx, cy), 5, (0, 0, 255), -1)
    cv.putText(img_display, f"({cx},{cy})", (cx + 10, cy - 10),
               cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    
    tx_point, ty_point = w//2 + tx, h//2 + ty
    cv.circle(img_display, (tx_point, ty_point), 5, (0, 255, 0), -1)
    cv.putText(img_display, f"({tx_point},{ty_point})", (tx_point + 10, ty_point - 10),
               cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv.imshow("Transformaciones", img_display)

    if cv.waitKey(1) & 0xFF == 27:
        break

cv.destroyAllWindows()


Usa los sliders para transformar la imagen.
Pulsa ESC para salir.
